In [1]:
import argparse
import math
import time

import torch
import torch.nn as nn
from net import gtnet
import numpy as np
import importlib

# from util import *
from data_prep import *
from trainer import Optim
import sys
import os

In [2]:
torch.__version__

'2.3.1+cu121'

In [3]:
torch.version.cuda

'12.1'

In [4]:
torch.cuda.is_available()

True

In [5]:
def evaluate(data, X, Y, model, evaluateL2, evaluateL1, batch_size, mean, std):
    model.eval()
    total_loss = 0
    total_loss_l1 = 0
    n_samples = 0
    predict = None
    test = None

    for X, Y in data.get_batches(X, Y, batch_size, False):
        X = torch.unsqueeze(X,dim=1)
        X = X.transpose(2,3)
        with torch.no_grad():
            output = model(X)
        output = torch.squeeze(output)
        if len(output.shape)==1:
            output = output.unsqueeze(dim=0)
        if predict is None:
            predict = output
            test = Y
        else:
            predict = torch.cat((predict, output))
            test = torch.cat((test, Y))

        total_loss += evaluateL2(mean + output * std, mean + Y * std).item()
        total_loss_l1 += evaluateL1(mean + output * std, mean + Y * std).item()
        n_samples += (output.size(0) * data.m)

    rse = math.sqrt(total_loss / n_samples) / data.rse
    rae = (total_loss_l1 / n_samples) / data.rae

    predict = (predict).data.cpu().numpy()
    Ytest = test.data.cpu().numpy()
    sigma_p = (predict).std(axis=0)
    sigma_g = (Ytest).std(axis=0)
    mean_p = predict.mean(axis=0)
    mean_g = Ytest.mean(axis=0)
    index = (sigma_g != 0)
    correlation = ((predict - mean_p) * (Ytest - mean_g)).mean(axis=0) / (sigma_p * sigma_g)
    correlation = (correlation[index]).mean()
    return rse, rae, correlation, predict

In [6]:
def train(data, X, Y, model, criterion, lr, wd, batch_size, clip, mean, std, cl, cl_step_size, seq_out_len):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    optimizer.zero_grad()
    total_loss = 0
    n_samples = 0
    task_level = 0
    
    iter = 0
    for X, Y in data.get_batches(X, Y, batch_size, True):
        model.zero_grad()
        X = torch.unsqueeze(X,dim=1)
        X = X.transpose(2,3)
        # Y = torch.unsqueeze(Y,dim=1)
        # Y = Y.transpose(2,3)
        # print(X.shape)
        if iter % args.step_size == 0:
            perm = np.random.permutation(range(args.num_nodes))
        num_sub = int(args.num_nodes / args.num_split)
#         print(f"num_sub:{num_sub}")
#         print(f"perm :{perm}") 

        for j in range(args.num_split):
            # print(f"j:{j}")
            if j != args.num_split - 1:
                id = perm[j * num_sub:(j + 1) * num_sub]
            else:
                id = perm[j * num_sub:]
                
#             print(f"id:{id}")
            id = torch.LongTensor(id).to(device)
            tx = X[:, :, id, :]
            ty = Y[:, :, id]
            output = model(tx,id)
            # print(f"output shape:{output.shape}")
            output = torch.squeeze(output)
            # print(f"output shape:{output.shape}")
            predictions = mean + output * std
            real_values = mean + ty * std

            if iter%cl_step_size==0 and task_level<=seq_out_len:
                task_level +=1
            if cl:
                loss = criterion(predictions[:, :, :task_level], real_values[:, :, :task_level])
            else:
                loss = criterion(predictions, real_values)

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            total_loss += loss.item()
            n_samples += (output.size(0) * data.m)
            grad_norm = optim.step()

        # if iter%100==0:
        #     print('iter:{:3d} | loss: {:.3f}'.format(iter,loss.item()/(output.size(0) * data.m)))
        iter += 1
        # print(f"The number of batches used are:{iter}")
        
    return total_loss / n_samples

In [7]:
parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')

parser.add_argument('--data', type=str, default='./data/solar_AL.txt',
                    help='location of the data file')
parser.add_argument('--log_interval', type=int, default=2000, metavar='N',
                    help='report interval')
parser.add_argument('--save', type=str, default='model/model.pt',
                    help='path to save the final model')
parser.add_argument('--optim', type=str, default='adam')
parser.add_argument('--L1Loss', type=bool, default=True)
parser.add_argument('--normalize', type=int, default=2)
parser.add_argument('--device',type=str,default='cuda:1',help='')
parser.add_argument('--gcn_true', type=bool, default=True, help='whether to add graph convolution layer')
parser.add_argument('--buildA_true', type=bool, default=True, help='whether to construct adaptive adjacency matrix')
parser.add_argument('--gcn_depth',type=int,default=2,help='graph convolution depth')
parser.add_argument('--num_nodes',type=int,default=137,help='number of nodes/variables')
parser.add_argument('--dropout',type=float,default=0.3,help='dropout rate')
parser.add_argument('--subgraph_size',type=int,default=20,help='k')
parser.add_argument('--node_dim',type=int,default=40,help='dim of nodes')
parser.add_argument('--dilation_exponential',type=int,default=2,help='dilation exponential')
parser.add_argument('--conv_channels',type=int,default=16,help='convolution channels')
parser.add_argument('--residual_channels',type=int,default=16,help='residual channels')
parser.add_argument('--skip_channels',type=int,default=32,help='skip channels')
parser.add_argument('--end_channels',type=int,default=64,help='end channels')
parser.add_argument('--in_dim',type=int,default=1,help='inputs dimension')
parser.add_argument('--seq_in_len',type=int,default=24*7,help='input sequence length')
parser.add_argument('--seq_out_len',type=int,default=1,help='output sequence length')
parser.add_argument('--horizon', type=int, default=3)
parser.add_argument('--layers',type=int,default=5,help='number of layers')

parser.add_argument('--batch_size',type=int,default=32,help='batch size')
parser.add_argument('--lr',type=float,default=0.0001,help='learning rate')
parser.add_argument('--weight_decay',type=float,default=0.00001,help='weight decay rate')

parser.add_argument('--clip',type=int,default=5,help='clip')

parser.add_argument('--propalpha',type=float,default=0.05,help='prop alpha')
parser.add_argument('--tanhalpha',type=float,default=3,help='tanh alpha')
parser.add_argument('--cl',type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to do curriculum learning')
parser.add_argument('--cl_step_size',type=int,default=3000,help='whether to do curriculum learning')
parser.add_argument('--epochs',type=int,default=1,help='')
parser.add_argument('--num_split',type=int,default=1,help='number of splits for graphs')
parser.add_argument('--step_size',type=int,default=20,help='step_size')

_StoreAction(option_strings=['--step_size'], dest='step_size', nargs=None, const=None, default=20, type=<class 'int'>, choices=None, required=False, help='step_size', metavar=None)

In [8]:
sys.argv = ['argparse_in_notebook','--data','.\SlowVaryingAmplitudes_23_30.csv', '--log_interval', '50', '--save', './models_buoy_data/model.pt',
            '--optim', 'adam','--L1Loss','True','--normalize','2',
           '--device','cuda','--gcn_true','True', '--buildA_true','True',
           '--gcn_depth','1', '--num_nodes','33', '--dropout','0.3', 
           '--subgraph_size', '1', '--node_dim', '40', '--dilation_exponential', '3', 
           '--conv_channels', '32', '--residual_channels', '64', '--skip_channels', '32', 
           '--end_channels', '64', '--in_dim', '1', '--seq_in_len', '10', 
           '--seq_out_len', '5', '--horizon', '3', '--layers', '4', 
           '--batch_size', '16', '--lr', '0.001', '--weight_decay', '0.0001', 
           '--clip', '5', '--propalpha','0.1', '--tanhalpha', '2', 
           '--epochs','500', '--num_split','1','--step_size','20', '--cl','True','--cl_step_size','5']

In [9]:
# Step 3: Parse the arguments
args = parser.parse_args()

In [10]:
# Access the arguments
print(f'data: {args.data}')
print(f'log_interval: {args.log_interval}')
print(f'save: {args.save}')
print(f'optim: {args.optim}')
print(f'L1Loss: {args.L1Loss}')
print(f'normalize: {args.normalize}')
print(f'device: {args.device}')
print(f'gcn_true: {args.gcn_true}')
print(f'buildA_true: {args.buildA_true}')
print(f'gcn_depth: {args.gcn_depth}')
print(f'num_nodes: {args.num_nodes}')
print(f'dropout: {args.dropout}')
print(f'subgraph_size: {args.subgraph_size}')
print(f'node_dim: {args.node_dim}')
print(f'dilation_exponential: {args.dilation_exponential}')
print(f'conv_channels: {args.conv_channels}')
print(f'residual_channels: {args.residual_channels}')
print(f'skip_channels: {args.skip_channels}')
print(f'end_channels: {args.end_channels}')
print(f'in_dim: {args.in_dim}')
print(f'seq_in_len: {args.seq_in_len}')
print(f'seq_out_len: {args.seq_out_len}')
print(f'horizon: {args.horizon}')
print(f'layers: {args.layers}')
print(f'batch_size: {args.batch_size}')
print(f'lr: {args.lr}')
print(f'weight_decay: {args.weight_decay}')
print(f'clip: {args.clip}')
print(f'propalpha: {args.propalpha}')
print(f'tanhalpha: {args.tanhalpha}')
print(f'epochs: {args.epochs}')
print(f'num_split: {args.num_split}')
print(f'step_size: {args.step_size}')
print(f'cl: {args.cl}')
print(f'cl_step_size: {args.cl_step_size}')

data: .\SlowVaryingAmplitudes_23_30.csv
log_interval: 50
save: ./models_buoy_data/model.pt
optim: adam
L1Loss: True
normalize: 2
device: cuda
gcn_true: True
buildA_true: True
gcn_depth: 1
num_nodes: 33
dropout: 0.3
subgraph_size: 1
node_dim: 40
dilation_exponential: 3
conv_channels: 32
residual_channels: 64
skip_channels: 32
end_channels: 64
in_dim: 1
seq_in_len: 10
seq_out_len: 5
horizon: 3
layers: 4
batch_size: 16
lr: 0.001
weight_decay: 0.0001
clip: 5
propalpha: 0.1
tanhalpha: 2.0
epochs: 500
num_split: 1
step_size: 20
cl: True
cl_step_size: 5


In [11]:
device = torch.device(args.device)
torch.set_num_threads(3)

In [12]:
device

device(type='cuda')

In [13]:
Data = DataLoaderMulti(args.data, 0.6, device, args.seq_in_len, args.seq_out_len)
mean = Data.train[2]
std = Data.train[3]

(1, 33)
The dataset shape is: (538, 33)
The input sequence length is: 10
The output sequence length is: 5
The train X-dataset shape is: torch.Size([308, 10, 33])
The validation X-dataset shape is: torch.Size([202, 10, 33])

The train y-dataset shape is: torch.Size([308, 5, 33])
The validation y-dataset shape is: torch.Size([202, 5, 33])



In [14]:
model = gtnet(args.gcn_true, args.buildA_true, args.gcn_depth, args.num_nodes,
                  device, dropout=args.dropout, subgraph_size=args.subgraph_size,
                  node_dim=args.node_dim, dilation_exponential=args.dilation_exponential,
                  conv_channels=args.conv_channels, residual_channels=args.residual_channels,
                  skip_channels=args.skip_channels, end_channels= args.end_channels,
                  seq_length=args.seq_in_len, in_dim=args.in_dim, out_dim=args.seq_out_len,
                  layers=args.layers, propalpha=args.propalpha, tanhalpha=args.tanhalpha, layer_norm_affline=False)
model = model.to(device)

In [15]:
print(args)
print('\nThe recpetive field size is', model.receptive_field)
nParams = sum([p.nelement() for p in model.parameters()])
print('Number of model parameters is', nParams, flush=True)

Namespace(data='.\\SlowVaryingAmplitudes_23_30.csv', log_interval=50, save='./models_buoy_data/model.pt', optim='adam', L1Loss=True, normalize=2, device='cuda', gcn_true=True, buildA_true=True, gcn_depth=1, num_nodes=33, dropout=0.3, subgraph_size=1, node_dim=40, dilation_exponential=3, conv_channels=32, residual_channels=64, skip_channels=32, end_channels=64, in_dim=1, seq_in_len=10, seq_out_len=5, horizon=3, layers=4, batch_size=16, lr=0.001, weight_decay=0.0001, clip=5, propalpha=0.1, tanhalpha=2.0, cl=True, cl_step_size=5, epochs=500, num_split=1, step_size=20)

The recpetive field size is 241
Number of model parameters is 764933


In [16]:
if args.L1Loss:
    criterion = nn.L1Loss(reduction="sum").to(device)
else:
    criterion = nn.MSELoss(reduction="sum").to(device)

evaluateL2 = nn.MSELoss(reduction="sum").to(device)
evaluateL1 = nn.L1Loss(reduction="sum").to(device)

In [17]:
best_val = 10000000
optim = Optim(model.parameters(), args.optim, args.lr, args.clip, lr_decay=args.weight_decay)

In [18]:
# from util import *
# train_dataloader = DataLoaderM(Data.train[0], Data.train[1], 16)
# train_dataloader.shuffle()
# for iter, (x, y) in enumerate(train_dataloader.get_iterator()):
#     trainx = torch.Tensor(x).to(device)
#     print(f"Train xsample size:{trainx.shape}")
#     trainx= trainx.transpose(1, 3)
#     print(f"Train xsample size after transposing:{trainy.shape}")
#     trainy = torch.Tensor(y).to(device)
#     print(f"Train y sample size:{trainx.shape}")
#     trainy = trainy.transpose(1, 3)
#     print(f"Train y sample size after transposing:{trainy.shape}")

In [19]:
# At any point you can hit Ctrl + C to break out of training early.
try:
    print('begin training')
    for epoch in range(1, args.epochs + 1):
        epoch_start_time = time.time()
        # train_loss = train(Data, Data.train[0], Data.train[1], model, criterion, optim, args.batch_size, args.clip, torch.Tensor(mean).to(device), torch.Tensor(std).to(device))
        train_loss = train(Data, Data.train[0], Data.train[1], model, criterion, args.lr, args.weight_decay, args.batch_size, args.clip, torch.Tensor(mean).to(device), torch.Tensor(std).to(device), args.cl, args.cl_step_size, args.seq_out_len)
        val_loss, val_rae, val_corr, val_predict = evaluate(Data, Data.valid[0], Data.valid[1], model, evaluateL2, evaluateL1, args.batch_size, torch.Tensor(mean).to(device), torch.Tensor(std).to(device))
        if epoch % 50 == 0:
            print('| end of epoch {:3d} | time: {:5.2f}s | train_loss {:5.4f} | valid rse {:5.4f} | valid rae {:5.4f} | valid corr  {:5.4f}'.format(
                    epoch, (time.time() - epoch_start_time), train_loss, val_loss, val_rae, val_corr), flush=True)
        # Save the model if the validation loss is the best we've seen so far.

        if val_loss < best_val:
            with open(args.save, 'wb') as f:
                torch.save(model, f)
            best_val = val_loss
        # if epoch % 50 == 0:
        #     test_acc, test_rae, test_corr, predictions = evaluate(Data, Data.test[0], Data.test[1], model, evaluateL2, evaluateL1, args.batch_size)
        #     print("test rse {:5.4f} | test rae {:5.4f} | test corr {:5.4f}".format(test_acc, test_rae, test_corr), flush=True)

except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

begin training
| end of epoch  50 | time:  1.09s | train_loss 0.0016 | valid rse 0.0473 | valid rae 0.1288 | valid corr  0.9798
| end of epoch 100 | time:  1.12s | train_loss 0.0012 | valid rse 0.0280 | valid rae 0.0766 | valid corr  0.9800
| end of epoch 150 | time:  1.13s | train_loss 0.0030 | valid rse 0.0321 | valid rae 0.0881 | valid corr  0.9807
| end of epoch 200 | time:  1.09s | train_loss 0.0011 | valid rse 0.0278 | valid rae 0.0770 | valid corr  0.9815
| end of epoch 250 | time:  1.10s | train_loss 0.0014 | valid rse 0.0368 | valid rae 0.0994 | valid corr  0.9827
| end of epoch 300 | time:  1.13s | train_loss 0.0007 | valid rse 0.0249 | valid rae 0.0680 | valid corr  0.9819
| end of epoch 350 | time:  1.14s | train_loss 0.0012 | valid rse 0.0313 | valid rae 0.0824 | valid corr  0.9828
| end of epoch 400 | time:  1.14s | train_loss 0.0011 | valid rse 0.0277 | valid rae 0.0722 | valid corr  0.9836
| end of epoch 450 | time:  1.13s | train_loss 0.0008 | valid rse 0.0369 | valid 

In [20]:
errors = []

vacc = []
vrae = []
vcorr = []
# acc = []
# rae = []
# corr = []
for i in range(10):
    with open(args.save, 'rb') as f:
        model = torch.load(f)

    vtest_acc, vtest_rae, vtest_corr, val_predictions = evaluate(Data, Data.valid[0], Data.valid[1], model, evaluateL2, evaluateL1, args.batch_size, torch.Tensor(mean).to(device), torch.Tensor(std).to(device))
    # test_acc, test_rae, test_corr, test_predictions = evaluate(Data, Data.test[0], Data.test[1], model, evaluateL2, evaluateL1, args.batch_size)

    print("validation rse {:5.4f} | validation rae {:5.4f} | validation corr {:5.4f}".format(vtest_acc, vtest_rae, vtest_corr))
    # print("final test rse {:5.4f} | test rae {:5.4f} | test corr {:5.4f}\n".format(test_acc, test_rae, test_corr))
    
    # val_acc, val_rae, val_corr, test_acc, test_rae, test_corr = vtest_acc, vtest_rae, vtest_corr, test_acc, test_rae, test_corr
    val_acc, val_rae, val_corr = vtest_acc, vtest_rae, vtest_corr
    vacc.append(val_acc)
    vrae.append(val_rae)
    vcorr.append(val_corr)
    # acc.append(test_acc)
    # rae.append(test_rae)
    # corr.append(test_corr)
print('\n\n')
print('10 runs average')
print('\n\n')
print("valid\trse\trae\tcorr")
print("mean\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.mean(vacc), np.mean(vrae), np.mean(vcorr)))
print("std\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.std(vacc), np.std(vrae), np.std(vcorr)))
print('\n\n')

errors.extend([np.mean(vacc), np.mean(vrae), np.mean(vcorr)])
np.savetxt(f'num_layers_{args.layers}.csv', errors, delimiter=',')
# print("test\trse\trae\tcorr")
# print("mean\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.mean(acc), np.mean(rae), np.mean(corr)))
# print("std\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.std(acc), np.std(rae), np.std(corr)))

validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0564 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0565 | validation corr 0.9844
validation rse 0.0203 | validation rae 0.0564 | validation corr 0.9844
validation rse 0.0204 | validation rae 0.0564 | validation corr 0.9844



10 runs average



valid	rse	rae	corr
mean	0.0204	0.0565	0.9844
std	0.0000	0.0000	0.0000





In [21]:
val_predictions.shape

(202, 5, 33)

In [22]:
import matplotlib.pyplot as plt
import random

fig, axes = plt.subplots(11,3, figsize=[35,25], dpi=300, sharex = True)
fig.suptitle("Multi step ahead predictions", fontsize=40)
plt.rcParams["font.family"]="serif"
plt.rcParams['font.size']=30

random_dataset = random.randint(0,Data.valid[0].shape[0])
for i in range(Data.valid[0].shape[2]):
    j = i//3
    k = i%3
    x_range = np.arange(0,len(Data.valid[0][random_dataset,:,i]))
    y_range = np.arange(len(Data.valid[0][random_dataset,:,i]),len(Data.valid[0][random_dataset,:,i])+len(Data.valid[1][random_dataset,:,i]))
    axes[j,k].plot(x_range, Data.valid[0][random_dataset,:,i], 'k-.',linewidth=2, label="x-data")
    axes[j,k].plot(y_range, Data.valid[1][random_dataset,:,i], color='b', marker = 'h', markersize = 3, label="truth")
    axes[j,k].plot(y_range, val_predictions[random_dataset,:,i], color='r', marker = 'h', markersize = 3, label='prediction')
    axes[j,k].set_title(f'a{i}')
    axes[j,k].tick_params(axis='both', which='major', labelsize=18)

    axes[j,k].legend(ncol = 3, fontsize = 14)

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.88, 
                    wspace=0.4, 
                    hspace=0.4)    
# plt.savefig(f"multi step ahead predictions_during training_num_layers_{args.layers}.png")
plt.show()